## Step 1: Import modules and packages, download reference docs

This goes through entire LastMile AI Eval flow from

1. create ingestion trace
2. generate a query + ground truth context pair per each node context in a document
  - taking those queries and running rag query traces to get actual retrieved context
3. listing query traces I want to include in a test set (defaults to last N queries for now)
4. create test Set with given query_traces, as well as storing the ground truth for the associated context for each query
5. create evaluation metrics based on ones provided by Llama Index
   - note: this is mainly from Llama Index, so the evaluation metrics are only focused on retrieval, nothing on outputs (though I store those as output events too)
6. create evaluation set by feeding these metrics with test set we just created

Some notes:
- no manual id grepping needed --> all taken care of by helper functions
probably needs to be better designed in future, just was focused on getting unblocked
- need to refactor ingestion_trace_id to map to trace-level, not marking rag query event level (right now it doesn't work, I'll add that later)
- some other small API convenience functions need to be added to the API, such as a helper function for `list_evaluation_sets()`


In [ ]:
# Install dependencies
# IMPORTANT: After running this cell, you MUST
# restart kernel for these changes to take effect

# !pip list | grep lastmile

# !pip3 install lastmile-eval #--upgrade --force-reinstall

!pwd

# Hacky way to locally install the lastmile-eval package lol
!pip3 install -e ../../../../..

!pip3 install llama-index

In [ ]:
!pip list | grep lastmile-eval

In [17]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.llms.openai import OpenAI

import pandas as pd

from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer


In [18]:
import os
import dotenv
# You can get your OPENAI_API_KEY from https://platform.openai.com/api-keys

dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LASTMILE_API_TOKEN = os.getenv("LASTMILE_API_TOKEN")

os.environ["LASTMILE_API_TOKEN"] = LASTMILE_API_TOKEN
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -o 'data/paul_graham/paul_graham_essay.txt'

## Step 2: Run and Trace Ingestion Pipeline

In [ ]:
# Instantiate a tracer object
tracer = get_lastmile_tracer("my_cool_tracer")


# You can use the tracer either as a decorator around a function (like below)
# or with the "with ... as span_variable_name:" syntax
@tracer.start_as_current_span("ingestion-root-span")
def run_ingestion_flow():
    documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

    # Register the doc file paths as a parameter
    doc_file_paths = [
        doc.metadata.get("file_path")
        for doc in documents
        if doc.metadata.get("file_path") is not None
    ]
    tracer.register_param("doc_file_paths", str(doc_file_paths))

    with tracer.start_as_current_span(
        "create-document-nodes"
    ) as _node_parser_span:
      # Register chunk_size as a parameter in this
      # trace's parameter set
      chunk_size = 512
      tracer.register_param("chunk_size", chunk_size)

      node_parser = SimpleNodeParser.from_defaults(chunk_size=chunk_size)
      nodes = node_parser.get_nodes_from_documents(documents)

    # Mark a RAG Ingestion trace event
    #   --> For now this only accepts strings and list of strings
    #   --> We can add more specific events (like what you'll see with
    #      the `mark_rag_query_trace_event` method) in the future
      tracer.mark_rag_ingestion_trace_event("Created document nodes!")
    with tracer.start_as_current_span(
        "embed-document-nodes"
    ) as _create_node_span:
        vector_index = VectorStoreIndex(nodes)
        query_engine = vector_index.as_query_engine()
        tracer.mark_rag_ingestion_trace_event("Created embeddings!")

    # We use these variables later in the notebook so need to return them
    # in this function
    return nodes, vector_index, query_engine


In [ ]:
# Run the ingestion flow and save the trace data
# This saves it to two tables:
# 1) The raw trace data that gets saved to Jaeger
# 2) The structured trace data that includes the paramSets, events,
#   etc that gets saved to our Postgres tables

# Run this cell once to generate an ingestion trace
nodes, vector_index, query_engine = run_ingestion_flow()

In [ ]:
# Let's print the trace data from Jaeger to
# show you what it looks like (search for "operationName" in the data)

from lastmile_eval.rag.debugger.tracing import (
    get_latest_ingestion_trace_id,
    get_trace_data,
)

ingestion_trace_id = get_latest_ingestion_trace_id()
get_trace_data(ingestion_trace_id)

In [ ]:
# Now let's fetch the trace event data from our postgres table
# Notice that the `traceId` column matches with the raw trace data

from lastmile_eval.rag.debugger.tracing import list_ingestion_trace_events

ingestion_trace_events = list_ingestion_trace_events(take=1)
pd.DataFrame.from_records(ingestion_trace_events["ingestionTraces"]).rename(  # type: ignore[fixme]
    columns={"id": "ragIngestionTraceEventId"}
)

## Part 3: Run and Trace Query Pipeline

In [24]:
import openai
from lastmile_eval.rag.debugger.api import (
    QueryReceived,
    ContextRetrieved,
    PromptResolved,
    LLMOutputReceived,
)

LLM_NAME = "gpt-4"

# Note, normally you can just call `query_engine.query(user_query)`
# but this abstracts away a lot of the steps so we will be doing
# each step manually to showcase how to use the tracer
PROMPT_TEMPLATE = """
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer:
"""


@tracer.start_as_current_span("query-root-span")
def run_query_flow(user_query: str, ingestion_trace_id: str):
    tracer.mark_rag_query_trace_event(
        QueryReceived(query=user_query), ingestion_trace_id
    )

    with tracer.start_as_current_span(
        "retrieve-context"
    ) as _retrieve_context_span:
        similarity_top_k = 5
        tracer.register_param("similarity_top_k", similarity_top_k)

        retriever = vector_index.as_retriever(
            similarity_top_k=similarity_top_k
        )
        retrieved_nodes = retriever.retrieve(user_query)
        retrieved_contexts = [node.get_text() for node in retrieved_nodes]

        retrieved_node_ids = [node.id_ for node in retrieved_nodes]
        tracer.register_param("retrieved_node_ids", retrieved_node_ids)

        tracer.mark_rag_query_trace_event(
            ContextRetrieved(context=retrieved_contexts), ingestion_trace_id
        )

    with tracer.start_as_current_span("resolve-prompt") as _resolve_prompt_span:
        resolved_prompt = PROMPT_TEMPLATE.replace(
            "{context_str}", "\n\n\n".join(retrieved_contexts)
        ).replace("{query_str}", user_query)
        tracer.mark_rag_query_trace_event(
            PromptResolved(fully_resolved_prompt=resolved_prompt),
            ingestion_trace_id,
        )

    with tracer.start_as_current_span("call-llm") as _llm_span:
        openai_client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        response = openai_client.chat.completions.create(
            model=LLM_NAME,
            messages=[{"role": "user", "content": resolved_prompt}],
        )
        output: str = response.choices[0].message.content
        tracer.mark_rag_query_trace_event(
            LLMOutputReceived(llm_output=output), ingestion_trace_id
        )


In [ ]:
# TODO: Right now the ingestion_trace_id within mark_rag_query_trace_event is
# no-op due to changes in assumptions, I'll fix later
run_query_flow("What did the author do growing up?", ingestion_trace_id)

In [ ]:
# Just like what we did with the ingestion trace,
# let's print out what this looks like in the PostGres data, as well as the
# pure trace data again
from lastmile_eval.rag.debugger.tracing import (
    list_query_trace_events,
)

query_trace_events = list_query_trace_events(take=1)
query_trace_events_df = pd.DataFrame.from_records(query_trace_events["queryTraces"]).rename(  # type: ignore[fixme]
    columns={"id": "ragQueryTraceEventsId"}
)
query_trace_events_df



In [ ]:
# This is what the trace data looks like
from lastmile_eval.rag.debugger.tracing import (
    get_trace_data,
)

query_trace_id = query_trace_events_df.iloc[0]["traceId"]
get_trace_data(query_trace_id)




## Part 4: Create Test Sets and Run Evaluators

In [ ]:
# NOTE: Running this cell on all the nodes will take a while (probably 5-10mins), so please be patient

# Change this to a lower value if you want to run faster
# If we use None, we will not use this value and use total_queries_per_batch
# instead
total_queries_to_run_override = (
    5  # None
)


# Ok we're now going to artifically generate a bunch of query + context
# (ground truth) pairs. We will then run the `run_query_flow()` method on these
# generated queries later

# Define an LLM
llm = OpenAI(model=LLM_NAME)


# This method `generate_question_context_pairs()` essentially
# calls an LLM to generate questions for us. See this URL for more details:
# https://github.com/run-llama/llama_index/blob/8b373239396134a92c9277b36aa7023c633c018a/llama-index-finetuning/llama_index/finetuning/embeddings/common.py#L49-L64
num_questions_per_chunk = 1
qa_dataset = generate_question_context_pairs(
    nodes[0:total_queries_to_run_override or len(nodes)],
    llm=llm,
    num_questions_per_chunk=num_questions_per_chunk
)

In [ ]:
# Run these queries through the `run_query_flow()` method

total_queries_per_batch = len(qa_dataset.queries)
total_queries_to_run = min(total_queries_to_run_override or total_queries_per_batch, total_queries_per_batch)

expected_node_ids: list[str] = []
for i, (query_id, query) in enumerate(qa_dataset.queries.items()):
    run_query_flow(query, ingestion_trace_id)
    associated_node_id_for_query = qa_dataset.relevant_docs[query_id]
    expected_node_ids.append(associated_node_id_for_query[0])

    print(f"Finished running {i+1}/{total_queries_to_run} queries...")
    if i + 1 == total_queries_to_run:
        break

# Have to reverse because the get_rag_query_trace_events() method
# returns the most recent trace events first
expected_node_ids.reverse()

In [ ]:
from lastmile_eval.rag.debugger.tracing import list_query_trace_events

query_trace_events = list_query_trace_events(take=total_queries_to_run)
query_trace_events_df = pd.DataFrame.from_records(query_trace_events["queryTraces"]).rename(  # type: ignore[fixme]
    columns={"id": "ragQueryTraceId"}
)
query_trace_events_df

## Part 5 - Run Evaluators from Query Trace Events data

This directly creates evaluation sets using the method `evaluate_rag_outputs()` without the need to create intermediate test cases and test sets. All you need is to define your query trace event rows in a dataframe.

In [ ]:
from lastmile_eval.rag.debugger.api.evaluation import evaluate_rag_outputs
from lastmile_eval.rag.debugger.tracing import (
    get_query_trace_event,
)

from lastmile_eval.text.metrics import calculate_rouge1_score
from llama_index.core.evaluation import HitRate, MRR


def rouge1(df: pd.DataFrame):
    return [
        # some weird error where it doesn't work for 0 values
        0.01 + x for x in calculate_rouge1_score(df["output"].tolist(), df["groundTruth"].tolist())
    ]

def extract_data_to_evaluate(
    row: pd.Series,
) -> tuple[list[str], list[str]]:
    trace_query_id: str = row["ragQueryTraceId"]
    trace_query_data = get_query_trace_event(trace_query_id)
    retrieved_node_ids = trace_query_data["paramSet"]["retrieved_node_ids"]
    expected_node_ids: list[str] = [row["groundTruth"]]
    return (retrieved_node_ids, expected_node_ids)


def compute_eval_score(
    retrieved_and_expected_node_ids_tuple: tuple[list[str], list[str]],
    evaluator: HitRate | MRR,
) -> float:
    retrieved_node_ids, expected_node_ids = (
        retrieved_and_expected_node_ids_tuple
    )
    return evaluator.compute(
        retrieved_ids=retrieved_node_ids, expected_ids=expected_node_ids
    ).score

# Example using a row-level function on the dataframe
def compute_mrr(df: pd.DataFrame):
    """
    We are demonstrating methods that are applied across a row instead of
    entire dataframe, such as the MRR and Hit Rate metrics from the 
    llama_index.core.evaluation package. In order to do this, we define a
    method at the row level where we:
    
    1. Extract the data to evaluate from the row
    2. Run the evaluators on this extracted data
    
    After that's done, we pass this row-level method to df.apply()
    """
    def evaluate_using_row_method(row: pd.Series) -> float:
        node_id_tuple = extract_data_to_evaluate(row)
        return compute_eval_score(node_id_tuple, MRR())
    
    return df.apply(evaluate_using_row_method, axis=1)

def compute_hit_rate(df: pd.DataFrame):
    """
    Another row-function example with hit_rate
    """
    def evaluate_using_row_method(row: pd.Series) -> float:
        node_id_tuple = extract_data_to_evaluate(row)
        return compute_eval_score(node_id_tuple, HitRate())
    
    return df.apply(evaluate_using_row_method, axis=1)
    
trace_level_evaluators = {
    "rouge1": rouge1,
    "mrr": compute_mrr,
    "hit_rate": compute_hit_rate,
}

# We must add groundTruth to the dataframe
query_trace_events_df["groundTruth"] = expected_node_ids

eval_result = evaluate_rag_outputs(
    project_id="can be anything for now",
    trace_level_evaluators=trace_level_evaluators,
    dataset_level_evaluators={},
    df=query_trace_events_df,
    lastmile_api_token=LASTMILE_API_TOKEN,
    evaluation_set_name="Cool new evaluation set name"
)

#print out result
eval_result

## Part 6 - Run Evaluators by creating intermediate test cases, test sets first

This is showing how to manually create test set 

In [ ]:
from lastmile_eval.rag.debugger.api import (
    create_test_set_from_rag_query_traces,
)

create_test_set_from_rag_query_traces(
    query_trace_events_df,
    test_set_name="Retrieval Eval Test Set",
    lastmile_api_token=LASTMILE_API_TOKEN,
    ground_truth=expected_node_ids,
)

In [ ]:
from lastmile_eval.rag.debugger.api import (
    get_latest_test_set_id,
)

test_set_id = get_latest_test_set_id()

In [ ]:
from lastmile_eval.rag.debugger.api import download_test_set
test_set_df = download_test_set(test_set_id)
test_set_df

In [37]:
from lastmile_eval.rag.debugger.tracing import (
    get_query_trace_event,
)

# Define some out of the box retrieval evaluators
# TODO: Set up some evaluators that also measure outputs too
"""
Hit Rate:
Hit rate calculates the fraction of queries where the correct answer is found
within the top-k retrieved documents. In simpler terms, it’s about how often
our system gets it right within the top few guesses.

Mean Reciprocal Rank (MRR):
For each query, MRR evaluates the system’s accuracy by looking at the rank of
the highest-placed relevant document. Specifically, it’s the average of the
reciprocals of these ranks across all the queries. So, if the first relevant
document is the top result, the reciprocal rank is 1; if it’s second, the
reciprocal rank is 1/2, and so on.
"""
from llama_index.core.evaluation import HitRate, MRR
hit_rate_evaluator = HitRate()
mrr_evaluator = MRR()
metric_evaluators = [hit_rate_evaluator, mrr_evaluator]

In [ ]:
# Manually doing it
data = []

def retrieved_correct_context_node(test_set_df):
    for _index, row in test_set_df.iterrows():
        trace_query_id = row["ragQueryTraceId"]
        trace_query_data = get_query_trace_event(trace_query_id)
        print(f"{trace_query_data=}")
        retrieved_node_ids = trace_query_data["paramSet"]["retrieved_node_ids"]
        expected_node_ids: list[str] = [row["groundTruth"]]

        evaluator_results = [
            evaluator.compute(
                retrieved_ids=retrieved_node_ids,
                expected_ids=expected_node_ids,
            ).score
            for evaluator in metric_evaluators
        ]
        data.append([trace_query_id, *evaluator_results])
    # trace_query_id = list_query_trace_events(take=1)["queryTraces"][0]["id"]


retrieved_correct_context_node(test_set_df)

import pandas as pd

columns = ["Trace Query Event Id", "Hit Rate", "MRR"]
eval_pd = pd.DataFrame(data, columns=columns)
eval_pd

In [ ]:
# Using the run_and_store_evaluations method
from lastmile_eval.rag.debugger.api import run_and_store_evaluations

result = run_and_store_evaluations(
    test_set_id,
    "Fake project name",
    {
        "Hit Rate": compute_hit_rate,
        "MRR": compute_mrr,
    },
    {},
    LASTMILE_API_TOKEN,
    f"Evaluation Results for Test Set {test_set_id}",
)

# TODO: Print out the evaluation test table with final evaluation metrics
result


In [ ]:
import requests
from requests import Response
from typing import Any, Optional

# TODO: Save this as it's own helper SDK from the lastmile-eval package
def list_evaluation_sets(
    take: int = 10,
    # TODO: Create macro for default timeout value
    timeout: int = 60,
) -> dict[str, Any]:  # TODO: Define eplicit typing for JSON response return
    """
    Get a list of evaluation sets from the LastMile API.

    Args:
        take: The number of evaluation sets to return. The default is 10.
        lastmile_api_token: The API token for the LastMile API. If not provided,
            will try to get the token from the LASTMILE_API_TOKEN
            environment variable.
            You can create a token from the "API Tokens" section from this website:
            https://lastmileai.dev/settings?page=tokens
        timeout: The maximum time in seconds to wait for the request to complete.
            The default is 60.

    Returns:
        A dictionary containing the evaluation sets.
    """
    lastmile_endpoint = f"https://lastmileai.dev/api/evaluation_sets/list?pageSize={str(take)}"

    response: Response = requests.get(
        lastmile_endpoint,
        headers={"Authorization": f"Bearer {LASTMILE_API_TOKEN}"},
        timeout=timeout,
    )
    # TODO: Handle response errors
    return response.json()

evaluation_sets = list_evaluation_sets(take=1)
evaluation_sets_df = pd.DataFrame.from_records(evaluation_sets["evaluationSets"]).rename(  # type: ignore[fixme]
    columns={"id": "evaluationSetId"}
)
pd.set_option('display.max_colwidth', None)
evaluation_sets_df

# TODO: evaluationSetMetrics looks a bit weird, should probalby have helper
# method to display it better, but it's ok for now